In [1]:
import os
import csv
import json
import pdfplumber
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
# Load environment variables
load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
# Function to extract text from PDF file
def extract_text_from_pdf(pdf_path):
    """
    Extracts text page-by-page from a PDF file using pdfplumber.
    Returns a list of strings, each representing the text of a page.
    """
    pages_text = []
    with pdfplumber.open(pdf_path) as pdf_file:
        for page in pdf_file.pages:
            text = page.extract_text()
            if text:
                pages_text.append(text)
    return pages_text

In [6]:
# test function

pages_text = extract_text_from_pdf("../data/raw/account/202204.pdf")
print(pages_text)
print(type(pages_text))
print(len(pages_text))

['JASON ROBINSON SEPULVEDA SALAZAR\njasonsepulvedas@gmail.com\nPANAMA PROVINCIA ABR/30/22\nRUC 47101-0002-306017 DV 99\nRESUMEN DE CUENTA BANCARIA\nDébitos Créditos Saldo Saldo Saldo\nNúmero de Cuenta Total Monto Total Monto Promedio Anterior a la Fecha\n114039365 12 5365.48 15 5512.72 3095.75 5520.40 5667.64\nDETALLE DE CUENTA\n114039365\nFecha N° de Referencia Concepto Débito Crédito Saldo\nSaldo Anterior 5,520.40\nABR/01 000038292 PAGO 402912******5075 2,747.03 2773.37\nABR/01 901404751 ACH BCOGENERAL Ixchel Anais de 110.00 2663.37\nABR/02 951460970 Pago DIGICEL CONTRATO 63067761 22.44 2640.93\nABR/02 951460976 Pago PANAPASS RECARGA 918587 30.00 2610.93\nABR/04 007430794 ACH CRE LAURA FABBIANA L 849.15 3460.08\nABR/04 000080804 PAGO 402912******5075 1,718.52 1741.56\nABR/05 007458632 ACH CRE LAURA FABBIANA L 243.05 1984.61\nABR/06 007465780 ACH CRE LAURA FABBIANA L 25.00 2009.61\nABR/06 007474090 ACH CRE LAURA FABBIANA L 38.61 2048.22\nABR/06 040700000 LIBERTY TECHNOLOGIES R PANAM 4

In [16]:
def call_openai_api_for_account_data_extraction(text):
    """
    Sends 'text' to the OpenAI model with instructions to extract
    banking transactions from credit card statement in a JSON format.

    Returns a list of dictionaries with the following keys:
        - date
        - reference
        - Description
        - debit (float)
        - credit (float)
        - balance (float)
    """
    # Define the system and user prompts
    system_prompt = (
        """You are an assistant that extracts banking transactions from text.
        Each transaction appears in a single line with these columns (in order):
        1. DATE
        2. REFERENCE
        3. DESCRIPTION
        4. DEBIT (float, can be 0.00 if not a debit)
        5. CREDIT (float, can be 0.00 if not a credit)
        6. BALANCE (float)

        Important details:
        - If the DEBIT column has a float greater than 0, the CREDIT column is 0.00 for that transaction.
        - If the CREDIT column has a float greater than 0, the DEBIT column is 0.00 for that transaction.
        - Negative values for debit or credit do not appear in the statement. Instead, 0.00 is used when the column does not apply.
        - The BALANCE column is always present at the end of the line.

        Example transaction lines:
        ABR/10 001800816 TEF DE:BRAIAN YAMIL DAGHERO WI  16.00 1989.27
        (meaning: date=ABR/10, reference=001800816, description="TEF DE:BRAIAN YAMIL DAGHERO WI", debit=0.00, credit=16.00, balance=1989.27)

        ABR/11 007410418 ACH CRE LAURA FABBIANA L   262.94 2252.21
        (meaning: date=ABR/11, reference=007410418, description="ACH CRE LAURA FABBIANA L", debit=0.00, credit=262.94, balance=2252.21)

        Instructions:
        1. Identify all lines that represent transactions with these columns.
        2. Return an array of JSON objects with the exact fields:
        - "date"
        - "reference"
        - "description"
        - "debit" (float)
        - "credit" (float)
        - "balance" (float)
        3. Return only the JSON, with no extra text or explanation.
        4. If there is ambiguity in a row, do your best to interpret it consistently with the column order: date, reference, description, debit, credit, balance. """
    )

    user_prompt = f"PDF text:\n\n{text}\n\nExtract the transactions in JSON."

    try:
        # Call the OpenAI API
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.0,
        )

        # Extract the data from the API response
        content = completion.choices[0].message.content
        transactions = json.loads(content)

        # return a list
        if isinstance(transactions, dict):
            transactions = [transactions]
        return transactions
    
    except Exception as e:
        print("Error when processing text with OpenAI:", e)
        return []

In [17]:
response = call_openai_api_for_account_data_extraction(pages_text[1])
print(response)
print(type(response))
print(len(response))

[{'date': 'ABR/10', 'reference': '001800816', 'description': 'TEF DE:BRAIAN YAMIL DAGHERO WI', 'debit': 16.0, 'credit': 0.0, 'balance': 1989.27}, {'date': 'ABR/11', 'reference': '007410418', 'description': 'ACH CRE LAURA FABBIANA L', 'debit': 0.0, 'credit': 262.94, 'balance': 2252.21}, {'date': 'ABR/12', 'reference': '001805913', 'description': 'TEF A : 104306337', 'debit': 532.0, 'credit': 0.0, 'balance': 1720.21}, {'date': 'ABR/12', 'reference': '041200221', 'description': 'PROTECCION ROBO 41432705260087', 'debit': 1.5, 'credit': 0.0, 'balance': 1718.71}, {'date': 'ABR/14', 'reference': '041500000', 'description': 'NETFLIX.COM 866-5', 'debit': 13.49, 'credit': 0.0, 'balance': 1705.22}, {'date': 'ABR/18', 'reference': '007479507', 'description': 'ACH CRE LAURA FABBIANA L', 'debit': 0.0, 'credit': 100.0, 'balance': 1805.22}, {'date': 'ABR/18', 'reference': '007489576', 'description': 'ACH CRE LAURA FABBIANA L', 'debit': 0.0, 'credit': 32.0, 'balance': 1837.22}, {'date': 'ABR/18', 'refe